# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397832909161                   -0.90    5.5   56.6ms
  2   -8.400253381668       -2.62       -1.74    1.0   17.8ms
  3   -8.400397926157       -3.84       -2.93    1.2   18.0ms
  4   -8.400427771215       -4.53       -2.90    3.2   23.1ms
  5   -8.400427763889   +   -8.14       -2.88    1.0   17.5ms
  6   -8.400428145218       -6.42       -4.65    1.0   17.4ms
  7   -8.400428155440       -7.99       -4.27    3.0   43.2ms
  8   -8.400428156249       -9.09       -5.66    1.0   18.0ms
  9   -8.400428156277      -10.55       -6.11    2.5   21.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397860479585                   -0.90           5.5    555ms
  2   -8.400384366540       -2.60       -1.79   0.80    2.0    513ms
  3   -8.400422735361       -4.42       -3.01   0.80    1.0    182ms
  4   -8.400428105794       -5.27       -3.40   0.80    2.2   18.4ms
  5   -8.400428152893       -7.33       -4.53   0.80    1.0   15.0ms
  6   -8.400428156239       -8.48       -5.76   0.80    2.8   19.2ms
  7   -8.400428156277      -10.42       -5.94   0.80    3.0   52.5ms
  8   -8.400428156277      -12.71       -6.97   0.80    1.0   15.6ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/8dE7C/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/8dE7C/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.629141041529                   -1.08    2.92s
  2   -1.661286302511        0.36       -0.70    114ms
  3   -4.486265583030        0.45       -0.41   87.4ms
  4   -5.852374881127        0.14       -0.50   39.4ms
  5   -7.558060635837        0.23       -0.70   38.9ms
  6   -7.718259674941       -0.80       -1.36   53.5ms
  7   -8.158719568990       -0.36       -1.60   29.3ms
  8   -8.234714036956       -1.12       -1.86   29.3ms
  9   -8.273483112864       -1.41       -2.02   29.0ms
 10   -8.317358502064       -1.36       -2.14   28

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397793313402                   -0.90    5.2   25.3ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427979362                   -1.78    26.2s
  2   -8.400428156277       -6.75       -4.02    2.03s
  3   -8.400428156277      -14.45       -7.81    111ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.215276473627085e-7
|ρ_newton - ρ_scfv| = 1.497337721348362e-7
|ρ_newton - ρ_dm|   = 2.005916443000951e-6
